In [18]:
# !pip install python-docx
# 0.8.11

In [78]:
# !pip install PyPDF2
# 2-2.12.1

In [79]:
# brew update
# brew install --cask libreoffice

In [104]:
import pandas as pd
import numpy as np
import json
import os
import re
import docx

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
import pickle

from pymystem3 import Mystem
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer

import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [40]:
# Проведем очистку данных: стемминг, единый регистр, удалим знаки препинания, цифры и т.п.

def preprocess(line):
    char_regex = re.compile(r'[^а-яa-z]')
    line = char_regex.sub(' ', line.lower())
    
    short_words = re.compile(r'\b[а-яa-z]{1,3}\b')
    line = short_words.sub(' ', line.lower())
    
    tokenized = toktok.tokenize(line)
    stemmed_str = []
    for i in tokenized:
        if i not in russian_stopwords:
            lemma = m.lemmatize(i)[0]
            stemmed_str.append(str(lemma))
#         stemmed = stemmer.stem(i)
#         stemmed_str += str(stemmed + ' ')
    return ' '.join(stemmed_str)

In [117]:
DOC_PATH = '/Users/mayabikmetova/xmas/hacka-aka-embedika/docs'

In [69]:
a = getText('hacka-aka-embedika/docs/03a70fe60be9ecc2a63798a361fc9689.docx')

In [19]:
m = Mystem()

In [23]:
russian_stopwords = nltk.corpus.stopwords.words('russian')

In [66]:
russian_stopwords.append('страница')

In [14]:
toktok = ToktokTokenizer()
stemmer = RussianStemmer()

In [41]:
preprocess(a)

'договор именовать дальнейший подрядчик лицо действующий основание именовать дальнейший заказчик лицо действующий основание заключать настоящий договор нижеследующий предмет договор подрядчик обязываться выполнять задание заказчик следующий работа указывать какой работа объем качество заказчик обязываться принимать результат работа оплачивать работа выполняться риск подрядчик заказчик срок способ выполнение работа материал предусматривать работа выполняться материал сила средство подрядчик подрядчик самостоятельно согласование заказчик определять способ выполнение задание заказчик подрядчик привлекать исполнение свой обязательство субподрядчик вправе вправе качество выполнять подрядчик работа должно соответствовать подробно описывать требование который должно соответствовать качество давать ссылка требование обычно предъявлять работа соответствующий род указывать какой документ образец эталон оригинал должно соответствовать качество подрядчик гарантировать качество выполнять работа теч

In [60]:
from PyPDF2 import PdfReader

reader = PdfReader("hacka-aka-embedika/docs/f57fe87f15a6dee2b17e804421be63b5.pdf")
# количество страниц
num_pages = reader.getNumPages()
text_pages = []

for i in range(num_pages):
    page = reader.pages[i].extract_text()
    text_pages.append(page)

In [63]:
pdf_text = ''.join(text_pages)

In [67]:
preprocess(pdf_text)

'договор купля продажа оборудование бывший употребление именовать дальнейший покупатель лицо действующий основание один сторона лицо действующий основание именовать дальнейший продавец сторона вместе отдельность именовать соответственно сторона сторона заключать настоящий договор далее договор нижеследующий предмет договор продавец обязываться передавать собственность далее оборудование товар комплектация согласно спецификация приложение договор покупатель обязываться прин опло тить оборудование порядок условие установленный настоящий дого вор также осуществлять вывоз товар территория продавец продавец гарантировать оборудование свобода правый третий закладывать обременять залог запрет состоять покупатель приобретать данный оборудование цель дальнейший разборка демонтаж обязательство сторона продавец обязываться производить ередача оборудование покупатель соответствие условие договор спецификация приложение договор одновременно оборудование передавать покупатель надлежащий товаросопров

In [87]:
!pwd

/Users/mayabikmetova/xmas


In [81]:
doc = 'hacka-aka-embedika/docs/f8eabec67b770c384d84f9feb7e02543.doc'

In [86]:
import subprocess

In [119]:
DOC_PATH

'/Users/mayabikmetova/xmas/hacka-aka-embedika/docs'

In [121]:
# конвертация doc в docx для дальнейшего прочтения

for FILE in os.listdir(DOC_PATH):
    if FILE.split('.')[1] == 'doc':
        subprocess.call(["cd {} && /Applications/LibreOffice.app/Contents/MacOS/soffice \
                         --headless --convert-to docx {} && rm {}".format(DOC_PATH, FILE, FILE)],
                         shell=True)
        print('Конвертирован файл ', FILE)


convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/02682d726b725f95b9ee85f751c043d0.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/02682d726b725f95b9ee85f751c043d0.docx using filter : Office Open XML Text
Конвертирован файл  02682d726b725f95b9ee85f751c043d0.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/2c81df29db63aebf495106881a52188f.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/2c81df29db63aebf495106881a52188f.docx using filter : Office Open XML Text
Конвертирован файл  2c81df29db63aebf495106881a52188f.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/31d24c0d4694e7c2fb19933b84414b40.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/31d24c0d4694e7c2fb19933b84414b40.docx using filter : Office Open XML Text
Конвертирован файл  31d24c0d4694e7c2fb19933b84414b40.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/57962abd8dbe0ce0c7056896ee4501f1.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/57962abd8dbe0ce

Конвертирован файл  49bce5af1c98cd017a310ea8edd3e55b.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/a0e06a957c1a729fe1d978605680e2ae.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/a0e06a957c1a729fe1d978605680e2ae.docx using filter : Office Open XML Text
Конвертирован файл  a0e06a957c1a729fe1d978605680e2ae.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/073a0d372820c3c2bffe9ba24a7ed7af.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/073a0d372820c3c2bffe9ba24a7ed7af.docx using filter : Office Open XML Text
Конвертирован файл  073a0d372820c3c2bffe9ba24a7ed7af.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/7ecd641f2ad81961c17455ed3ebeb2ab.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/7ecd641f2ad81961c17455ed3ebeb2ab.docx using filter : Office Open XML Text
Конвертирован файл  7ecd641f2ad81961c17455ed3ebeb2ab.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/2060962a8b7ca4c45dc041c3f624225a.doc -> /Users/m

Конвертирован файл  41b36e9a90f36be9ed2a2b4f44e5c848.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/5908cb5da047d6c9e6dfea6337fb3189.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/5908cb5da047d6c9e6dfea6337fb3189.docx using filter : Office Open XML Text
Конвертирован файл  5908cb5da047d6c9e6dfea6337fb3189.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/d143c89d002fcef3e2bd2efdb4966f55.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/d143c89d002fcef3e2bd2efdb4966f55.docx using filter : Office Open XML Text
Конвертирован файл  d143c89d002fcef3e2bd2efdb4966f55.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/bdfb262c00991dddebfc5d4a87604284.doc -> /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/bdfb262c00991dddebfc5d4a87604284.docx using filter : Office Open XML Text
Конвертирован файл  bdfb262c00991dddebfc5d4a87604284.doc
convert /Users/mayabikmetova/xmas/hacka-aka-embedika/docs/8540a7dccb3a9e4d0a1a2611e077ce9d.doc -> /Users/m

In [124]:
for FILE in os.listdir(DOC_PATH):
    if FILE.split('.')[1] != 'doc' and FILE != '.DS_Store':
        print(FILE)

910ccf7c9b029846e4390ddac5f8d687.docx
69ab7557dee21939aa7432b23a54cb2b.docx
bec0aa38d1383172690a18d16b07f154.docx
31d24c0d4694e7c2fb19933b84414b40.docx
dda3ea2dc3a078682620f926a54aa0bf.docx
cbf3217807791a37499a7b8d94616fdc.docx
8ee681886de97951748528c7c78d03a7.docx
2054a58154349bc54b691666fe3f89b1.pdf
f8eabec67b770c384d84f9feb7e02543.docx
e3f3df754bdf7578a71b7a9fa1500ad9.rtf
b40a9d048b199d5f4db62a6a2335f2a0.pdf
3588ff6fb8f5c5a5f70196500198ca3a.docx
bdfb262c00991dddebfc5d4a87604284.docx
f57fe87f15a6dee2b17e804421be63b5.pdf
4db6b233fda895c3bffcb5fdc5b8e1de.rtf
5e9754740f6c89a96cf5996404ec3c27.docx
137eaaa047f557233943012897e46cec.docx
8b82f3c800e486d9da9a13c98f7a40d6.docx
7a120d4c7cec157324effa671e2d0812.docx
7aff676ead9fe323b2c542e60accb1ae.docx
2c81df29db63aebf495106881a52188f.docx
0f7f507d0af90aba3c35484de016d8b4.docx
97d4cf03b88c9e040557ab1b61e2e7a2.docx
242b138fd2025b7a5207876098355e03.docx
855c8450351c98a25ea2ac22a0256c7f.docx
2351cf256df1b683a425211c1ed0e848.docx
259b4813668421411

In [113]:
with open('hacka-aka-embedika/classes.json', 'r') as f:
    data = json.load(f)
    
labels = pd.DataFrame({'filename': data.keys(), 'class': data.values()})

In [126]:
labels['extension'] = labels['filename'].apply(lambda x: x.split('.')[1])

In [127]:
labels

,filename,class,extension
0,5908cb5da047d6c9e6dfea6337fb3189.doc,Договоры для акселератора/Договоры поставки,doc
1,14711e4fc8e56f0c75856c8837ec04cb.doc,Договоры для акселератора/Договоры поставки,doc
2,7eb67b5aecf3f3190aab0a5f8ea32172.docx,Договоры для акселератора/Договоры поставки,docx
3,b40a9d048b199d5f4db62a6a2335f2a0.pdf,Договоры для акселератора/Договоры поставки,pdf
4,84fec112d02288861e7af59f468131fb.docx,Договоры для акселератора/Договоры поставки,docx
...,...,...,...
115,f57fe87f15a6dee2b17e804421be63b5.pdf,Договоры для акселератора/Договоры купли-продажи,pdf
116,1ea8809d696a4bd6a2076fbc6fd28c23.doc,Договоры для акселератора/Договоры купли-продажи,doc
117,57962abd8dbe0ce0c7056896ee4501f1.doc,Договоры для акселератора/Договоры купли-продажи,doc
118,2c81df29db63aebf495106881a52188f.doc,Договоры для акселератора/Договоры купли-продажи,doc


In [115]:
labels['class'].unique()

array(['Договоры для акселератора/Договоры поставки',
       'Договоры для акселератора/Договоры оказания услуг',
       'Договоры для акселератора/Договоры подряда',
       'Договоры для акселератора/Договоры аренды',
       'Договоры для акселератора/Договоры купли-продажи'], dtype=object)

In [128]:
labels['extension'].unique()

array(['doc', 'docx', 'pdf', 'rtf'], dtype=object)